<a href="https://colab.research.google.com/github/kcckcc/AudioPlayer/blob/main/Stella_Bible_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Find the .mp3 file under recording_path not matching _part?.mp3

import os
import re

def find_mp3_file(recording_path):
  for filename in os.listdir(recording_path):
    if filename.endswith(".mp3") and not re.match(r".*_part\d\.mp3", filename):
      return os.path.join(recording_path, filename)
  return None


recording_path = "D:\\React\\biblestudy\\public\\book\\Proverbs (2015)\\2015-05-02"
mp3_file = find_mp3_file(recording_path)
if mp3_file:
  print(f"Found MP3 file: {mp3_file}")
else:
  print("No matching MP3 file found.")

!D:\React\biblestudy\DownloadRecording\SplitAudio\bin\Release\net8.0\SplitAUdio.exe mp3_file

In [ ]:

!D:\React\biblestudy\DownloadRecording\SplitAudio\bin\Release\net8.0\SplitAUdio.exe mp3_file

In [ ]:
# prompt: Iterate the file in current folder and store the name end with "_part?.mp3" in an array and sort with ascending order

import os
import re

def get_sorted_mp3_files(directory):
  mp3_files = []
  try:
    for filename in os.listdir(directory):
      if re.match(r".*_part\d+\.mp3", filename):  # Use regex for more flexible matching
        mp3_files.append(filename)
    mp3_files.sort() # Sort the files alphabetically
    return mp3_files
  except OSError as e:
    print(f"Error accessing directory: {e}")
    return []


mp3_part_files = get_sorted_mp3_files(recording_path)

if mp3_part_files:
  print("MP3 Part files:")
  for file in mp3_part_files:
    print(file)
else:
  print("No matching MP3 files found in the directory or an error occurred.")

# Get book and verses name
file_name = mp3_part_files[0]

idx = file_name.rfind('_part')  # Find the index of the first '.' (file extension)
if idx != -1:
    recording_name = file_name[:idx]
    idx = recording_name.rfind('_')
    if idx != -1:
        book_name = recording_name[:idx].replace('_', ' ')
        verses = recording_name[idx + 1:].replace('^', ':')
        print(f"Book Name: {book_name}")
        print(f"Verses: {verses}")
    else:
        print("'_' not found in filename")
else:
    print("'.' not found in filename")


In [33]:
# prompt: Define a function that write response_text to an output file and upload to client

import pathlib

def write_and_upload(response_text, output_filename):
  text_bytes = response_text.encode('utf-8')
  text_path = pathlib.Path(output_filename)
  text_path.write_bytes(text_bytes)
  file_upload = client.files.upload(file=text_path)


In [34]:
%pip install -U -q 'google-genai>=1.4.0' # 1.4.0 is needed for chat history

from google.colab import userdata
from google import genai
from google.genai import types

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [35]:
MODEL_ID = "gemini-2.5-pro-exp-03-25" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

In [36]:
from IPython.display import Markdown

system_instruction = """
  You are an expert in language translation and a bible scholar.
"""

chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
)

chat = client.chats.create(
    model=MODEL_ID,
    config=chat_config,
)

In [ ]:
mp3_part_files = ["Proverbs_10^7-14_part1.mp3", "Proverbs_10^7-14_part2.mp3"]

for mp3_part_file in mp3_part_files:
  print (mp3_part_file)


In [45]:
# prompt: Invoke client.models.generate_context and check for error such as daily limit exceed

from google.api_core import exceptions

def generate_context_with_error_handling(client, model_id, messages):
    try:
        response = client.models.generate_context(
            model=model_id,
            context=messages
        )
        return response
    except exceptions.PermissionDenied as e:
        print(f"Permission denied: {e}")
        if "daily limit" in str(e).lower():
            print("Daily limit exceeded for this model.")
        return None
    except exceptions.ResourceExhausted as e:
        print(f"Resource exhausted: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [69]:
import time

def PromptModel(prompt, input_data):
    try:
        start_time = time.time()
        response = client.models.generate_content(
        model=MODEL_ID,
          contents=[
            input_data,
            prompt,
          ],
        )
        end_time = time.time()
        elapsed_time = end_time - start_time
        minutes = int(elapsed_time // 60)
        seconds = int(elapsed_time % 60)
        print(f"Response took {minutes} minutes and {seconds} seconds to complete.")
        return response.text
    except exceptions.PermissionDenied as e:
        print(f"Permission denied: {e}")
        if "daily limit" in str(e).lower():
            print("Daily limit exceeded for this model.")
        return "limit_error"
    except exceptions.ResourceExhausted as e:
        print(f"Resource exhausted: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


def PromptModelWithRetry(prompt, input_data):
  retry_count = 0
  response = None
  while response is None and retry_count < 30:
    response = PromptModel(prompt, input_data)
    if response == "limit_error":
      print("Daily limit exceeded for this model.")
      response = None
      break
    elif response is not None:
        print(f"response size {len(response)}.")
        return response
    else:
      print(f"Retrying {mp3_part_file}... Attempt {retry_count + 1}/30")
      time.sleep(60)  # Wait for 1 minute
      retry_count += 1

  return response


In [ ]:

# prompt:  For each mp3_part_files, call ConvertToTranscript, stop if response is "limit_error",  wait for 1 minute and retry if response is None for 30 times, otherwise append response to cantonese_text

import time
prompt = "Translate this Cantonese bible study " + book_name + " " + verses + " teaching class audio to Chinese Traditional character. Ignore laughing sound. After that, remove timeline and group message into paragraphs. Do not summarize the message. She mention Saddleback Church a lot."

cantonese_text = ""
for mp3_part_file in mp3_part_files:
  print("Try  " + mp3_part_file)
  # response = ConvertToTranscript(os.path.join(recording_path, mp3_part_file))
  input_data = client.files.upload(file=mp3_part_file)
  response = PromptModelWithRetry(prompt, input_data)
  if response is not None:
    cantonese_text += response

if cantonese_text:
  write_and_upload(cantonese_text, 'cantonese.txt')
else:
  print("No text to write.")

write_and_upload(client, cantonese_text, 'cantonese.txt')


Try  Proverbs_10^7-14_part1.mp3


In [ ]:
prompt = "Translate this to Simplified Chinese characters. Add translation for any English in Simplified Chinese characters next to it. Keep the paragraph format and add a line spacing between paragraphs."

simplified_text = PromptModelWithRetry(prompt, cantonese_text)
write_and_upload( response.text, 'simplified.txt')

In [ ]:
prompt = "Translate to English with all Chinese words inside the translation removed. Keep the paragraph format and add a line spacing between paragraphs. Also fix grammatical errors such as repeated words."

english_text = PromptModelWithRetry(prompt, cantonese_text)
write_and_upload( response.text, 'english.txt')

In [ ]:
prompt = "Summarize the teaching in Cantonese."

cantonese_summary = PromptModelWithRetry(prompt, cantonese_text)
write_and_upload( cantonese_summary, 'cantonese_summary.txt')

In [ ]:
prompt = "Translate this to Simplified Chinese characters.  Keep the paragraph format and add a line spacing between paragraphs."

simplified_summary = PromptModelWithRetry(prompt, cantonese_summary)
write_and_upload( simplified_summary, 'simplified_summary.txt')

In [62]:
prompt = "Translate this to English with all Chinese words inside the translation removed. Keep the paragraph format and add a line spacing between paragraphs."
english_summary = PromptModelWithRetry(prompt, cantonese_summary)
write_and_upload( english_summary, 'english_summary.txt')

cache_tokens_details=None cached_content_token_count=None candidates_token_count=5041 candidates_tokens_details=None prompt_token_count=1283 prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=1283)] thoughts_token_count=3639 tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=6324 traffic_type=None
